In [1]:
import pandas as pd
import numpy as np
import math
from os import path
from PIL import Image
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
import matplotlib.pyplot as plt

In [2]:
df1=pd.read_csv('Nigeria2019_ACLED_Extract.csv')
df1.columns

Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')

In [ ]:
df2=pd.read_csv('Nigeria2019_Newsfeed.csv')
df2.columns

In [ ]:
#Q1B

In [ ]:
df1

In [ ]:
math.isnan(df1.actor2[0])

In [ ]:
actors=set(df1.actor1)|set(df1.assoc_actor_1)|set(df1.actor2)|set(df1.assoc_actor_2)
actor_cols=['actor1','actor2','assoc_actor_1','assoc_actor_2']
event_count={}
fatality_count={}
for actor in actors:
    event_count[actor]=0
    fatality_count[actor]=0

In [ ]:
for i in range(len(df1)):
    for actor in actor_cols:
        a=df1[actor][i]
        event_count[a]+=1
        fatality_count[a]+=df1.fatalities[i]

In [ ]:
x=list(event_count.keys())[1]

In [ ]:
event_count.pop(x)
fatality_count.pop(x)

In [ ]:
print(len(event_count),len(fatality_count))

In [ ]:
A=list(event_count.keys())
B=list(event_count.values())
C=list(fatality_count.values())

In [ ]:
df=pd.DataFrame(list(zip(A,B,C)),columns=['Actor','No. of events','No. of fatalities'])
df

In [ ]:
df[df['No. of events']>200]

In [ ]:
df.to_csv('1B.csv')

In [ ]:
#Q1A

In [ ]:
events=set(df1.event_type)
event_count={}
for event in events:
    event_count[event]=0

for i in range(len(df1)):
    event_count[df1.event_type[i]]+=1
        
event_count

In [ ]:
df=pd.DataFrame(list(zip(event_count.keys(),event_count.values())),columns=['Event type','Count'])
df

In [ ]:
df.to_csv('1A.csv')

In [ ]:
df1

In [ ]:
#Q1C

In [ ]:
#df1=df1[df1.event_type!='Strategic developments']

In [ ]:
locations=set(df1.location)
fatality_count={}
lat={}
long={}

for location in locations:
    fatality_count[location]=[0,0,0]

for i in range(len(df1)):
    if df1.event_type[i]!='Strategic developments':
        fatality_count[df1.location[i]][1]=df1.latitude[i]
        fatality_count[df1.location[i]][2]=df1.longitude[i]
        fatality_count[df1.location[i]][0]+=df1.fatalities[i]
        
fatality_count

In [ ]:
df=pd.DataFrame(list(zip(fatality_count.keys(),fatality_count.values())),columns=['Location','Total fatalities'])
df

In [ ]:
df['Latitude']=df['Total fatalities'].map(lambda x:x[1])
df['Longitude']=df['Total fatalities'].map(lambda x:x[2])
df['Total fatalities']=df['Total fatalities'].map(lambda x:x[0])
df=df[df.Latitude!=0]
df

In [ ]:
sum(df['Total fatalities'])

In [ ]:
df[df['Total fatalities']>=70]

In [ ]:
df.to_csv('1C.csv')

In [ ]:
#Q1D

In [ ]:
stopwords=list(STOPWORDS)+['mins','ago','say','says','new']
wordcloud = WordCloud(stopwords=stopwords).generate(' '.join(list(df2.Newsfeed_Description)))

plt.imshow(wordcloud,interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#Q3

In [ ]:
event_types=list(set(df1.event_type))
var_Explosions_or_Remote_violence=0
var_Protests=0
var_Battles=0
var_Riots=0
var_Violence_against_civilians=0

for i in range(len(df2)):
    description=df2.Newsfeed_Description2[i]
    
    if 'explosion' in description or 'remote violence' in description:
        var_Explosions_or_Remote_violence+=1
    elif 'battle' in description:
        var_Battles+=1
    elif 'riot' in description:
        var_Riots+=1
    elif 'protest' in description:
        var_Protests+=1
    elif 'violence' in description and 'civilians' in description:
        var_Violence_against_civilians+=1
    
print(var_Explosions_or_Remote_violence,var_Protests,var_Battles,var_Riots,var_Violence_against_civilians)

In [ ]:
df2

In [ ]:
l1=[87,622,545,183,660]
l2=[712,784,341,101,24]
from scipy.stats.stats import pearsonr  
pearsonr(l1,l2)

In [ ]:
#additional analysis - violence against civilians over the months

In [ ]:
df1['Month']=df1['event_date'].map(lambda x:x[-6:-3])
viol_df=df1[df1.event_type=='Violence against civilians']

In [ ]:
for month in list(set(viol_df.Month)):
    print(month,len(viol_df[viol_df.Month==month]))

In [ ]:
#dditional analysis - Involvement of police vs military

In [9]:
police='Police Forces of Nigeria (2015-)'
military='Military Forces of Nigeria (2015-)'

police_df=df1[(df1.actor1==police)|(df1.actor2==police)|(df1.assoc_actor_1==police)|(df1.assoc_actor_2==police)]
military_df=df1[(df1.actor1==military)|(df1.actor2==military)|(df1.assoc_actor_1==military)|(df1.assoc_actor_2==military)]

In [ ]:
milpo=pd.DataFrame(columns=['Event type','Military events','Police events'])
milpo

In [ ]:
for event in list(set(df1.event_type)):
    len_po=len(police_df[police_df.event_type==event])
    len_mil=len(military_df[military_df.event_type==event])
    milpo.loc[len(milpo)]=[event,len_mil,len_po]

In [ ]:
milpo

In [ ]:
milpo.to_csv('extra1.csv')

In [ ]:
#

In [10]:
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

In [11]:
milpo1=pd.DataFrame(columns=['Month','Military events','Police events'])
milpo1

,Month,Military events,Police events


In [12]:
police_df['Month']=police_df['event_date'].map(lambda x:x[-6:-3])
military_df['Month']=military_df['event_date'].map(lambda x:x[-6:-3])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
police_df['Month']=police_df['event_date'].map(lambda x:x[-6:-3])
military_df['Month']=military_df['event_date'].map(lambda x:x[-6:-3])

for month in months:
    len_po=len(police_df[police_df.Month==month])
    len_mil=len(military_df[military_df.Month==month])
    milpo1.loc[len(milpo1)]=[month,len_mil,len_po]

In [14]:
milpo1

,Month,Military events,Police events
0,Jan,62,15
1,Feb,39,14
2,Mar,23,25
3,Apr,25,28
4,May,40,16
5,Jun,39,8
6,Jul,26,18
7,Aug,22,16
8,Sep,26,25
9,Oct,27,13


In [17]:
milpo1.to_csv('extra2.csv')